# NOTE: 

### This notebook may be out of date. Please look at commissioning_design_example_all_skies.ipynb or commissioning_design_example_spectrophotometric_standards.ipynb for a more up-to-date example

# Commisioning Design Example - All Skies

In this example notebook, we will be showing how to build an all sky design (as may be done during commisioning) using Mugatu.

Below, we will import the desired packages and connect to targetdb.

In [1]:
from sdssdb.peewee.sdss5db import targetdb
import os
import roboscheduler.cadence as cadence
from mugatu.comm_designs import all_sky_design_RS


targetdb.database.connect_from_parameters(user='sdss',
                                          host='localhost',
                                          port=7500)

True

Next, we have to add the cadences using roboscheduler. For this example I am loading an old list of cadences from robostrategy:

In [2]:
cadence.CadenceList().fromfits(filename='rsCadences-test-newfield-apo.fits')

Now we can create a design of all skies.

In [3]:
from mugatu.comm_designs import ObsTime
from astropy.time import Time

# ra center of the field
racen = 100.
#dec center of the field
deccen = 45.
# position angle of the field
position_angle = 24.
# observatory of observation
observatory = 'APO'
# Julian Date of the observation
# if exact date is not know, we can use ObsTime to estimate
# the best viewing time for a given LST
obsTime = Time(ObsTime().nominal(lst=racen)).jd
# number of apogee skies desired in the design
n_sky_apogee = 150
# number of boss skies desired in the design
n_sky_boss = 350
# the name of the cadence for the design
cadence = 'bright_single-1'

des, fps_design = all_sky_design_RS(racen=racen,
                                    deccen=deccen,
                                    position_angle=position_angle,
                                    observatory=observatory,
                                    obsTime=obsTime,
                                    n_sky_apogee=n_sky_apogee,
                                    n_sky_boss=n_sky_boss,
                                    cadence=cadence)

[WARNING]: Warning! Wavelengths not supplied, defaulting to 6231 angstrom (CoordIOUserWarning)


The above function returns a Robostrategy Field object and a Mugatu FPSDesign object (with the Robostrategy assignments). To assess if Robostrategy was able to achieve the desired number of skies requested we can check by:

In [4]:
print(des.assess())

Field cadence: bright_single-1

Calibration targets:
 sky_boss (want 350): 304
 standard_boss (want 80): 0
 sky_apogee (want 150): 150
 standard_apogee (want 20): 0

Science targets:
 BOSS targets assigned: 0
 APOGEE targets assigned: 0
 Targets per epoch: 0

Carton completion:




If we want to re-validate the design with Mugatu, we can run the following:

In [5]:
fps_design.validate_design()

[WARNING]: Some targets could not be assigned to fiber (MugatuWarning)
[WARNING]: Some targets removed from design due to collisions (MugatuWarning)


As we can see from the warnings, some of the targets were not able to be assigned to a robot in Mugatu. To check which catalogids were not able to be assigned (either due to the target not being in range of a robot or due to collisions), you can run:

In [6]:
print(fps_design.targets_unassigned), len(fps_design.targets_unassigned)

[7610556048, 7610553593, 7610553403, 7610553382, 7610556071, 7610552781, 7610556448, 7610554104, 7610556542, 7610553810, 7610554249, 7610554060, 7610560325, 7610554565, 7610554560, 7610554669, 7610553321, 7610553310, 7610553066, 7610553394, 7610553345, 7610552672, 7610553644, 7610553598, 7610553609, 7610553468, 7610552735, 7610553946, 7610554162, 7610553745, 7610553951, 7610553962, 7610554892, 7610554021, 7610554087, 7610554850, 7610560045, 7610554771, 7610554529, 7610554744, 7610554461, 7610554490, 7610554497, 7610554271, 7610554402]


(None, 45)

In [7]:
print(fps_design.targets_collided), len(fps_design.targets_collided)

[7610556400, 7610553791, 7610553508, 7610554141, 7610554185, 7610558032, 7610554624, 7610554396, 7610556618, 7610561085, 7610560329, 7610555584, 7610555700]


(None, 13)

These types of discrepencies between validations are simply due to how robostrategy coverts (ra,dec) to FPS (x,y) compared to Mugatu. To check this, we can adjust the FPSDesign object as we like in order to change the (x,y) positions to the robostrategy calculated ones:

In [8]:
import kaiju

ev = eval("(fps_design.design['x'] != -9999.99)")

fps_design.design['x'][ev], fps_design.design['y'][ev] = des.radec2xy(ra=fps_design.design['ra'][ev],
                                                                      dec=fps_design.design['dec'][ev])

After resetting the robot grid, we can then re-validate the design with these new (x,y) coordinates:

In [9]:
# reset the grid
fps_design.rg = kaiju.robotGrid.RobotGridFilledHex()

for k in fps_design.rg.robotDict.keys():
    fps_design.rg.homeRobot(k)

fps_design.targets_unassigned = []
fps_design.targets_collided = []

In [10]:
# re-validate the design
fps_design.validate_design()

Now we can check the status of any unassigned robots and see that we find none:

In [11]:
print(fps_design.targets_unassigned), len(fps_design.targets_unassigned)

[]


(None, 0)

In [12]:
print(fps_design.targets_collided), len(fps_design.targets_collided)

[]


(None, 0)